In [9]:
import numpy as np
import cv2
import heapq
import statistics
import math

In [10]:
def popularity(image,k):
        (m,n,_) = image.shape
        d = {}
        for i in range(m):
            for j in range(n):
                t = tuple(image[i,j])
                if t in d:
                    d[t] += 1
                else:
                    d[t] = 1
        top_k_colors =heapq.nlargest(k, d, key=d.get)
        return top_k_colors

In [11]:
def split(list_of_tuples,colors,color_to_split,k):
    if (k==1):
        return [sum([x[0]*x[1] for x in list_of_tuples])/(sum([x[1] for x in list_of_tuples]))]
    median = statistics.median([x[0][color_to_split] for x in list_of_tuples])
    left = filter(lambda x: x[0][color_to_split] <= median , list_of_tuples)
    right = filter(lambda x: x[0][color_to_split] > median, list_of_tuples)
    return (split(left,colors,(color_to_split+1)%colors,k/2)+split(right,colors,(color_to_split+1)%colors,k/2))

In [12]:
def median_cut(image,k):
    (m,n,c) = image.shape
    d = {}
    for i in range(m):
        for j in range(n):
            t = tuple(image[i,j])
            if t in d:
                d[t] += 1
            else:
                d[t] = 1
    l = d.values()
#     print(type(l))
    # color to split missing
    split(l,c,k)

In [13]:
def popularity_quant(image, k):
    finalImage = image.copy()
    color_map = popularity(image, k)
    (m,n,_) = image.shape
        
    for i in range(m):
        for j in range(n):
            t = tuple(image[i,j])
            min_dist = 10000.0
            for col in color_map:
                dist = get_norm(t, col)
                if min_dist > dist :
                    min_dist = dist
                    min_col = col
                    
            finalImage[i,j] = np.asarray(min_col)
    return finalImage

In [14]:
def get_norm(t1 , t2):
    (xa, ya, za) = t1
    (xb, yb, zb) = t2
    return math.sqrt((xa-xb)^2 + (ya-yb)^2 + (za-zb)^2)


In [15]:
# test_image = cv2.imread('test1.png', cv2.IMREAD_GRAYSCALE)
test_image = cv2.imread('test1.png')
cv2.imshow('test_image',test_image)
cv2.waitKey()
cv2.destroyAllWindows()

In [18]:
test_image.shape

(540, 470, 3)

In [16]:
img = popularity_quant(test_image, 10)
cv2.imshow('Popularity Cut image',img)
cv2.waitKey()
cv2.destroyAllWindows()

/home/udit01/anaconda3/envs/imquant/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: overflow encountered in ubyte_scalars
  after removing the cwd from sys.path.


In [17]:
cv2.imwrite('popularity_test1.png', img)

True

In [20]:
print(cv2.COLORMAP_BONE)

1


In [86]:
print(tuple(test_image[0,0]))

(246, 246, 243)


In [8]:
help(filter)

Help on class filter in module builtins:

class filter(object)
 |  filter(function or None, iterable) --> filter object
 |  
 |  Return an iterator yielding those items of iterable for which function(item)
 |  is true. If function is None, return the items that are true.
 |  
 |  Methods defined here:
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __iter__(self, /)
 |      Implement iter(self).
 |  
 |  __next__(self, /)
 |      Implement next(self).
 |  
 |  __reduce__(...)
 |      Return state information for pickling.
 |  
 |  ----------------------------------------------------------------------
 |  Static methods defined here:
 |  
 |  __new__(*args, **kwargs) from builtins.type
 |      Create and return a new object.  See help(type) for accurate signature.



In [ ]:
def get_error(t1, t2):
    (xa, ya, za) = t1
    (xb, yb, zb) = t2
    return ((xa-xb)^2 , (ya-yb)^2 , (za-zb)^2 )

In [ ]:
def dither(image, color_map):
    # Assuming we have the color map already
    finalImage = image.copy()
    (m,n,_) = image.shape
        
    for i in range(m):
        for j in range(n):
            t = tuple(image[i,j])
            min_dist = 10000.0
            for col in color_map:
                dist = get_norm(t, col)
                if min_dist > dist :
                    min_dist = dist
                    min_col = col
                    
            finalImage[i,j] = np.asarray(min_col)
            # is the error formula correct ?
            error = get_error(t, min_col)
            if(j+1 < n):
                finalImage[i,j+1] = finalImage[i, j+1] + error*(3.0/8.0)
            if(i+1 < m):
                finalImage[i+1,j] = finalImage[i+1, j] + error*(3.0/8.0)
            if(j+1 < n and i+1 < m):
                finalImage[i+1,j+1] = finalImage[i+1, j+1] + error*(1.0/4.0)
 
    return finalImage